<a href="https://colab.research.google.com/github/vincentbriat/Super-resolution-investigation/blob/main/Discriminator_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Checks if the code is in a colab notebook

try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False

Run on Colab notebook

In [ ]:
if IN_COLAB:
  !pip install basicsr
  drive.mount('/content/drive/')
  !unzip /content/drive/MyDrive/Datasets/DIV2K_valid_HR.zip
  !unzip /content/drive/MyDrive/Datasets/DIV2K_valid_LR_clean.zip
  !unzip /content/drive/MyDrive/Datasets/DIV2K_train_LR_clean.zip
  #!unzip /content/drive/MyDrive/Datasets/DIV2K_train_HR.zip
  !unzip /content/drive/MyDrive/ML/Indiv_Project/Second_Year/Pruning/80_percent/gen_images_0.zip
  FOLDER_LR_TEST = 'DIV2K_valid_LR_clean'
  FOLDER_HR_TEST = 'DIV2K_valid_HR'
  FOLDER_LR_TRAIN = 'DIV2K_train_LR_clean'
  #FOLDER_HR_TRAIN = 'DIV2K_train_HR'
  FOLDER_GEN_IMAGES = 'gen_images_0'

  STUDENT_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/student.pth'
  STUDENT_RECORDS_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/student.csv'
  GENERATOR_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/generator.pth'
  GENERATOR_RECORDS_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/generator.csv'
  DISCRIMINATOR_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/Pruning/80_percent/discriminator.pth'
  DISCRIMINATOR_RECORDS_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/Pruning/80_percent/discriminator.csv'

  TEACHER_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/ESRGAN_models/RealESRGAN_x4plus.pth'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.9/305.9 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 20.3 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214837 sha256=e7384491fa90b844890d50726aca827f149a5923b54d168b4c4a2f4ff6b15929
  Stored in directory: /root/.cache/pip/wheels/42/d6/3d/6d8c4b8fabeb93dd442eb255f133e8928c75d903fe20e9472c
Successfully built basicsr
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successf

Run on my Windows desktop

In [ ]:
if not IN_COLAB:
  FOLDER_LR_TEST = 'D:\Downloads\Div2k\DIV2K_valid_LR_clean'
  FOLDER_HR_TEST = 'D:\Downloads\Div2k\DIV2K_valid_HR'
  FOLDER_LR_TRAIN = 'D:\Downloads\Div2k\DIV2K_train_LR_clean'
  FOLDER_HR_TRAIN = 'D:\Downloads\Div2k\DIV2K_train_HR'
  
  STUDENT_MODEL_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\student.pth'
  STUDENT_RECORDS_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\student.csv'
  GENERATOR_MODEL_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\generator.pth'
  GENERATOR_RECORDS_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\generator.csv'
  DISCRIMINATOR_MODEL_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\discriminator.pth'
  DISCRIMINATOR_RECORDS_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\discriminator.csv'
  
  TEACHER_MODEL_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\KD\ESRGAN_models\RealESRGAN_x4plus.pth'

# Setup

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import cv2
import csv
import math
import numpy as np
import os
import queue
import threading
from basicsr.archs.rrdbnet_arch import RRDBNet
import torchvision
from os import listdir, environ, path
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models import vgg19

# Making sure to use the gpu, if available
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_device(torch.device(0))

In [128]:
class DIV2KLoader(object):
  def __init__(self,  low_res_folder, low_res_size: int = 48, high_res_folder = None, preprocessing_input = None, preprocessing_output = None, include_targets = True):
    self.low_res_folder = low_res_folder
    self.high_res_folder = high_res_folder
    self.img_names = sorted(listdir(low_res_folder))
    self.include_targets = include_targets
    self.len = len(self.img_names)
    self.crop = torchvision.transforms.RandomCrop(low_res_size*4)
    self.resize = torchvision.transforms.Resize(low_res_size)
    
    if include_targets or high_res_folder is not None:
      self.target_names = sorted(listdir(high_res_folder))

    if preprocessing_input is None:
      self.preprocessing_input = torchvision.transforms.ToTensor()
    else:
      self.preprocessing_input = preprocessing_input

    if preprocessing_output is None:
      self.preprocessing_output = torchvision.transforms.ToTensor()
    else:
      self.preprocessing_output = preprocessing_output
  
  def __getitem__(self, i):
    # Get the right image
    img = Image.open(Path(self.low_res_folder).joinpath(self.img_names[i]))

    if self.include_targets and self.high_res_folder:
      target = Image.open(Path(self.high_res_folder).joinpath(self.target_names[i]))
      return self.preprocessing_input(img), self.preprocessing_output(target)
    
    # if include_targets is True and there's only one set of images
    elif self.include_targets and not self.high_res_folder:
      target = self.crop(self.preprocessing_input(img))
      return self.resize(target), target

    else:
      return self.preprocessing_input(img)

  def __len__(self):
    return self.len
  
  def restrict_size(self, size):
    if size < len(self.img_names) and size > 0:
      self.len = size
    else:
      self.len = len(self.img_names)
      print(f"Size must be between 0 and {len(self.img_names)}")

class Loader_D(DIV2KLoader):
  def __init__(self, fake_folder, crop_size: int, real_folder = None):
    super().__init__(fake_folder)
    self.img_names = sorted(listdir(fake_folder))
    self.fake_folder = fake_folder
    self.real_folder = real_folder
    self.len = len(self.img_names)
    self.crop_size = crop_size
    self.include_targets = False
    
    if real_folder:
      self.include_targets = True
      self.target_names = sorted(listdir(real_folder))
    
  def __getitem__(self, i):
    # Get the right image
    img = Image.open(Path(self.fake_folder).joinpath(self.img_names[i]))
    img = torchvision.transforms.ToTensor()(img)


    if self.include_targets:
      target = Image.open(Path(self.real_folder).joinpath(self.target_names[i]))
      target = torchvision.transforms.ToTensor()(target)

      x, y, h, w = torchvision.transforms.RandomCrop.get_params(img, output_size=(self.crop_size, self.crop_size))

      img = torchvision.transforms.functional.crop(img, x, y, h, w)
      target = torchvision.transforms.functional.crop(target, x, y, h, w)

      return img, target
    else:
      return img

In [ ]:
# 0 is resize
# 1 is crop center
# 2 is crop random (requires finish implementation)

MODE = 1
IM_SIZE = 50

def resize_ratio(img, size=IM_SIZE):
  width, height = img.size
  if width > height:
    width, height = size, int(height * size / width)
  else:
    width, height = int(width * size / height), size
  return width, height

def preprocessing_output(img):
  im_w, im_h = img.size
  if MODE == 1:
    img = img.crop(((im_w - 400)/2, (im_h - 400)/2, (im_w + 400)/2, (im_h + 400)/2))
  elif MODE == 2:
    width, height = resize_ratio(img)
    origin_x, origin_y = 0, 0
    img = img.crop((origin_x, origin_y, origin_x + width*4, origin_y + height*4))
  return torchvision.transforms.ToTensor()(img).to(DEVICE)

def preprocessing_input(img):
  im_w, im_h = img.size
  if MODE == 1:
    img = img.crop(((im_w - 400)/2, (im_h - 400)/2, (im_w + 400)/2, (im_h + 400)/2)).resize((100, 100))
  elif MODE == 2:
    width, height = resize_ratio(img)
    origin_x, origin_y = 0, 0
    img = img.crop((origin_x, origin_y, origin_x + width*4, origin_y + height*4)).resize((width, height))
  return torchvision.transforms.ToTensor()(img).to(DEVICE)

data_test = DIV2KLoader(FOLDER_LR_TEST)
data_test.restrict_size(10)
dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=1, shuffle=False)

data_train = DIV2KLoader(FOLDER_LR_TRAIN)
dataloader_train = torch.utils.data.DataLoader(data_train, batch_size=1, shuffle=False)

# Discriminator

In [165]:
# http://ethereon.github.io/netscope/#/gist/db945b393d40bfa26006

class ConvBlock(nn.Module):
  def __init__(self, in_channels: int, out_channels: int, kernel_size: int, use_act: bool = True, **kwargs):
    super().__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs)
    self.bn = nn.BatchNorm2d(out_channels)
    self.activation = nn.ReLU(inplace = True) if use_act else nn.Identity()

  def forward(self, x):
    return self.activation(self.conv(x))

class ResidualBlock_v1(nn.Module):
  def __init__(self, in_channels):
    super().__init__()
    self.block = nn.Sequential(
        ConvBlock(in_channels, in_channels, 3, stride=1, padding=1),
        ConvBlock(in_channels, in_channels, 3, stride=1, padding=1))

  def forward(self, x):
    return x + self.block(x)

class ResidualBlock_v2(nn.Module):
  def __init__(self, in_channels, mid_channels, **kwargs):
    super().__init__()
    self.block = nn.Sequential(ConvBlock(in_channels, mid_channels, 1, padding=0, **kwargs),
                               ConvBlock(mid_channels, mid_channels, 3, padding=1, stride=1),
                               ConvBlock(mid_channels, mid_channels*4, 1, padding=0, stride=1, use_act = False))
    
    if in_channels == mid_channels*4:
      self.residual = nn.Identity()
    else:
      self.residual = ConvBlock(in_channels, mid_channels*4, 1, padding=0, stride=1 if in_channels == mid_channels else 2, use_act = False)

  def forward(self, x):
    return self.block(x) + self.residual(x)

class ResNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.head = nn.Sequential(ConvBlock(3, 64, 7, stride=2), nn.MaxPool2d(3, 2))
    self.body = nn.Identity()
    self.tail = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
                              nn.Flatten(),
                              nn.Linear(2048,1))
    
  def forward(self, x):
      return self.tail(self.body(self.head(x)))

class ResNet_18(ResNet):
  def __init__(self):
    super().__init__()
    architecture = [64, 128, 256, 512]
    self.body = nn.Sequential(*[self._makeBlock(channels) for channels in architecture])

  def _makeBlock(self, in_channels):
    return nn.Sequential(ResidualBlock_v1(in_channels), ResidualBlock_v1(in_channels))

class ResNet_50(ResNet):
  def __init__(self):
    super().__init__()
    architecture = [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3
    self.body = self._makeBlock(architecture)
  
  def _makeBlock(self, architecture):
    layers = [ResidualBlock_v2(64, 64, stride=1)]

    for i in range(1, len(architecture)):
      mid_channels = architecture[i]
      in_channels = architecture[i-1] * 4
      layers.append(ResidualBlock_v2(in_channels, mid_channels, stride=1 if architecture[i-1] == architecture[i] else 2))

    return nn.Sequential(*layers)

# Follows the guidelines of https://github.com/soumith/ganhacks
class Discriminator(nn.Module):
  def __init__(self, model: nn.Module):
    super().__init__()
    self.model = model
    self.loss_fn = nn.BCELoss()
    self.optimizer = torch.optim.Adam(self.model.parameters())
    self.cur_epoch = 0

  def forward(self, x):
    return self.model(x)

  def train_one_epoch(self, dataloader):
    self.model.train()
    cumu_loss = 0
    accuracy = 0
    # This dataloader outputs (Generator(LR_images), HR_images)
    for batch, (X, y) in enumerate(dataloader):
      fake, real = X.to(DEVICE), y.to(DEVICE)
      
      out = torch.cat([fake, real])
      out = torch.sigmoid(self.model(out))
      target = torch.cat([torch.zeros((fake.shape[0], 1)), torch.ones((real.shape[0], 1))])

      loss = self.loss_fn(out, target)
      cumu_loss += loss.item()
      
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
      
      out = (out > 0).float()
      accuracy_tmp = torch.count_nonzero(out == target) / target.shape[0]
      accuracy += accuracy_tmp

      if batch % 2 == 0:
        print(f'Trained batch {batch} with loss {loss.item()}')
        print(f'Accuracy: {accuracy_tmp}')

    return {'loss': cumu_loss/len(dataloader), 'accuracy': accuracy/len(dataloader)}

  def run_training(self, dataloader, epochs, path_record:str = None, path_model: str = None, load_epoch: bool = True):
    # Initializes the records file if not existant
    
    if path_record and not Path(path_record).is_file():
      self.initialize_csv(path_record)
      self.cur_epoch = 0
    elif load_epoch:
      # Loads the next epoch to continue training
      self.load_cur_epoch(path_record)

    while self.cur_epoch < epochs:
      result = self.train_one_epoch(dataloader)
      if path_record:
        self.record(path_record, [self.cur_epoch, result['loss'], result['accuracy']])
      if path_model:
        self.save(path_model)
      self.cur_epoch += 1

  def load_cur_epoch(self, path):
    with open(path) as records_file:
      epoch = print(records_file.readlines()[-1].split(',')[0])
      self.cur_epoch = int(epoch) + 1
  
  def load(self, path):
    saved_info = torch.load(path, map_location=torch.device(DEVICE))
    self.model.load_state_dict(saved_info['model_state_dict'])
    self.optimizer.load_state_dict(saved_info['optimizer_state_dict'])
    #self.scheduler.load_state_dict(saved_info['lr_scheduler_state_dict'])
    self.loss_fn = saved_info['loss_fn']

  def save(self, path):
    torch.save({'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'loss_fn': self.loss_fn,
                'lr_scheduler_state_dict': 0},
               path)
    print("Discriminator saved Successfully!")

  def initialize_csv(self, path):
    self.record(path, ['Epoch', 'Loss', 'Accuracy'])
    print("Records created Successfully!")

  def record(self, path, row):
    with open(path, 'a', newline='') as f:
      csv.writer(f).writerow(row)
    print("Discriminator recorded Successfully!")

class VGGLoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.vgg = vgg19(pretrained=True).features[:35].eval().to(DEVICE)

    for param in self.vgg.parameters():
        param.requires_grad = False

    self.loss = nn.MSELoss()

  def forward(self, input, target):
    vgg_input_features = self.vgg(input)
    vgg_target_features = self.vgg(target)
    return self.loss(vgg_input_features, vgg_target_features)

class GAN:
  def __init__(self, gene, disc):
    self.disc = disc
    self.gen = gene
    self.opt_d = torch.optim.Adam(self.disc.parameters(), lr=2e-5)
    self.opt_g = torch.optim.Adam(self.gen.parameters(), lr=2e-5)
    self.lmb = 5e-3
    self.nu = 1e-2
    self.vgg_loss = VGGLoss()
    self.l1_loss = nn.L1Loss()
    
  def run_training(self, dataloader):
    cum_loss_d, cum_loss_g = 0, 0
    accuracy = 0
    for batch, (X, y) in enumerate(dataloader):
      # Train discriminator
      fake = self.gen(X.to(DEVICE))
      y_cuda = y.to(DEVICE)

      loss_g = self.train_one_model(y_cuda, fake, True)
      loss_d = self.train_one_model(y_cuda, fake, False)

      cumu_loss_d += loss_d
      cumu_loss_g += loss_g

      if batch % 20 == 0:
        print(f'Training batch {batch} with discriminator loss {loss_d:.2f} and generator loss {loss_g:.2f}')
      
      out_fake, out_real = torch.sigmoid(fake), torch.sigmoid(y_cuda)
      accuracy_tmp = torch.count_nonzero(out_fake < 0) / out_fake.shape[0] + torch.count_nonzero(out_real > 0) / out_real.shape[0]
      accuracy += accuracy_tmp
      #print(f'Accuracy: {accuracy_tmp}')

    return {'discriminator': cumu_loss_d / len(dataloader),'generator': cumu_loss_g / len(dataloader), 'accuracy': accuracy / len(dataloader)}

  def train_one_model(self, y_cuda, fake, is_generator: bool):
    d_ra_real, d_ra_fake = self.out_ra(self.disc(y_cuda), self.disc(fake if is_generator else fake.detach()))
    loss = self.loss_g(d_ra_real, d_ra_fake, fake, y_cuda) if is_generator else self.loss_d(d_ra_real, d_ra_fake)

    optimizer = self.opt_g if is_generator else self.opt_d
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

  def out_ra(self, out_real, out_fake):
    return torch.sigmoid(out_real - torch.mean(out_fake)), torch.sigmoid(out_fake - torch.mean(out_real))

  def loss_d(self, d_ra_real, d_ra_fake):
    return -torch.mean(torch.log(d_ra_real)) - torch.mean(torch.log(1 - d_ra_fake))

  def loss_g(self, d_ra_real, d_ra_fake, fake, real):
    loss_percep = self.vgg_loss(fake, real)
    loss_ra = self.loss_d(d_ra_fake, d_ra_real)
    loss_l1 = self.l1_loss(fake, real)

    return loss_percep + self.lmb * loss_ra + self.nu * loss_l1

In [166]:
disc = Discriminator(ResNet_50())
disc.load(DISCRIMINATOR_MODEL_PATH)

data_d = Loader_D(FOLDER_GEN_IMAGES, 20, real_folder= FOLDER_LR_TRAIN)
data_d.restrict_size(2)
dataloader_d = torch.utils.data.DataLoader(data_d, batch_size=16, shuffle=False)

In [167]:
disc.run_training(dataloader_d, 20, path_record= DISCRIMINATOR_RECORDS_PATH, path_model= DISCRIMINATOR_MODEL_PATH, load_epoch = False)

Trained batch 0 with loss 1.46455717086792
Accuracy: 0.5
Discriminator recorded Successfully!
Discriminator saved Successfully!
Trained batch 0 with loss 0.4755561649799347
Accuracy: 0.5
Discriminator recorded Successfully!
Discriminator saved Successfully!
Trained batch 0 with loss 0.3496401607990265
Accuracy: 0.5
Discriminator recorded Successfully!


KeyboardInterrupt: ignored